In [25]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import re
from torch.utils.data import Dataset, random_split, DataLoader
from transformers import BertTokenizer
import unicodedata
import py_vncorenlp
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/home/duypd/ThisPC-DuyPC/SG-Retrieval/khangTA/vncorenlp')

2024-11-30 17:07:13 INFO  WordSegmenter:24 - Loading Word Segmentation model


In [10]:
data_frame = pd.read_csv('/home/duypd/ThisPC-DuyPC/khangta-thesis/Dataset/final_data.csv')
data_frame

,TRICH_YEU,NAME,HINHTHUC
0,"ĐƠN RÚT HỒ SƠ CỦA LÂM VĂN HỒNG, P. TA",Khác,Công văn
1,"V/v góp ý dự thảo quy tắc ứng xử của CB,CC,VC ...",Khác,Công văn
2,V/V PHỐI HỢP PHƯƠNG ÁN PCLB NĂM 2012,Khác,Công văn
3,Ông (bà) Lê Ngọc Kim Loan,Khác,Công văn
4,chấn chỉnh tình hình thu nộp Quỹ PCLB đối tượn...,Khác,Công văn
...,...,...,...
101238,v/v thông qua dự thảo Phương án bồi thường&com...,NaN,Giấy mời
101239,Liên quan việc bàn giao hồ sơ chuyển mục đích ...,NaN,Giấy mời
101240,Đến tham dự buổi làm việc và thu thập hồ sơ có...,Khác,Giấy mời
101241,Vv trao đổi nội dung liên quan quy trình tiếp ...,Khác,Giấy mời


In [39]:
def preprocess_sentence(sentence):
    sentence = sentence.strip()
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = re.sub(r'[^\w\s.,!?_]', '', sentence)
    sentence.replace('comma','')
    return sentence

In [40]:
data_frame_1 = data_frame[["TRICH_YEU", "HINHTHUC"]]
data_frame_1

,TRICH_YEU,HINHTHUC
0,"ĐƠN RÚT HỒ SƠ CỦA LÂM VĂN HỒNG, P. TA",Công văn
1,"V/v góp ý dự thảo quy tắc ứng xử của CB,CC,VC ...",Công văn
2,V/V PHỐI HỢP PHƯƠNG ÁN PCLB NĂM 2012,Công văn
3,Ông (bà) Lê Ngọc Kim Loan,Công văn
4,chấn chỉnh tình hình thu nộp Quỹ PCLB đối tượn...,Công văn
...,...,...
101238,v/v thông qua dự thảo Phương án bồi thường&com...,Giấy mời
101239,Liên quan việc bàn giao hồ sơ chuyển mục đích ...,Giấy mời
101240,Đến tham dự buổi làm việc và thu thập hồ sơ có...,Giấy mời
101241,Vv trao đổi nội dung liên quan quy trình tiếp ...,Giấy mời


In [41]:
ht = data_frame_1['HINHTHUC']
ty = data_frame_1['TRICH_YEU']

ht_new = []
ty_new = []
for x, y in zip(ht, ty):
    ht_new.append(x)
    y = preprocess_sentence(y)
    y = rdrsegmenter.word_segment(y)
    ty_new.append(preprocess_sentence(y[0]))

In [42]:
ty_new

['ĐƠN RÚT HỒ_SƠ CỦA LÂM VĂN HỒNG , P. TA',
 'Vv góp_ý dự_thảo quy_tắc ứng_xử của CB , CC , VC quận 12',
 'VV PHỐI_HỢP PHƯƠNG_ÁN PCLB NĂM 2012',
 'Ông_bà Lê_Ngọc_Kim_Loan',
 'chấn_chỉnh tình_hình thu nộp Quỹ_PCLB đối_tượng doanh_nghiệp tại các quận_huyện',
 'Giải_quyết đơn của ông Võ_Văn_Lý',
 'cấp số căn_hộ khu nhà ở xã_hội cho LLVT QK7 chung_cư Đông_hải',
 'VV RÀ_SOÁT , TÌM VỊ_TRÍ XÂY_DỰNG VĂN_PHÒNG KHU_PHỐ 7 P. TA',
 'TRẢ_LỜI ĐƠN CỦA TRẦN HƯỜNG , P.TTN',
 'Thống_nhất hướng tuyến điều_chỉnh của các đoạn tuyến cáp ngầm 220kV trên đường Tô_Ký thuộc công_trình Cải_tạo đường_dây 220kV Phú_Lâm_Hóc_Môn 1 thành 2 mạch',
 'ông Nguyễn_Huy_Cường , phường Tân_Hưng_Thuận vv báo_cáo xác_minh liên_quan đến hồ_sơ đăng_ký cấp GCN nhà_đất',
 'Vv cung_cấp thông_tin',
 'v.v trả_lời ý_kiến phản_ánh của nhân_dân qua đường_dây_nóng Thành_phố',
 'Vê viêc quyêt toan chi phi xây dưng nha tiêp dân tra xe vi pham ATGT cua công_an quân .',
 'VỀ TRẢ_LỜI KIẾN_NGHỊ MỘT_SỐ KHÓ_KHĂN , VƯỚNG_MẮC TRONG QUÁ_TRÌNH THỰC_H

In [46]:
ty_new = [i.replace("comma", "") for i in ty_new]

In [47]:
ty_new[101238]

'vv thông_qua dự_thảo Phương_án bồi thường hỗ_trợ và tái định_cư và kế_hoạch thực_hiện dự_án đầu_tư xây_dựng công_trình nạo vét kiên_cố_hoá rạch Bà The đoạn từ cầu Mụ_Duyên đến rạch Giao Khẩu P.TX. Lúc 16h ngày 16122022'

In [50]:
df = pd.DataFrame({
    "HINHTHUC": ht_new,
    "TRICH_YEU": ty_new
})

df.to_csv("final_seg_words.csv", index=False, encoding="utf-8-sig")

In [49]:
df

,HINHTHUC,TRICH_YEU
0,Công văn,"ĐƠN RÚT HỒ_SƠ CỦA LÂM VĂN HỒNG , P. TA"
1,Công văn,"Vv góp_ý dự_thảo quy_tắc ứng_xử của CB , CC , ..."
2,Công văn,VV PHỐI_HỢP PHƯƠNG_ÁN PCLB NĂM 2012
3,Công văn,Ông_bà Lê_Ngọc_Kim_Loan
4,Công văn,chấn_chỉnh tình_hình thu nộp Quỹ_PCLB đối_tượn...
...,...,...
101238,Giấy mời,vv thông_qua dự_thảo Phương_án bồi thường hỗ_t...
101239,Giấy mời,Liên_quan việc bàn_giao hồ_sơ chuyển mục_đích ...
101240,Giấy mời,Đến tham_dự buổi làm_việc và thu_thập hồ_sơ có...
101241,Giấy mời,Vv trao_đổi nội_dung liên_quan quy_trình tiếp_...


In [52]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")


In [67]:
sentence = 'Chúng_tôi là những nghiên_cứu_viên .'  
input_ids = torch.tensor([tokenizer.encode(sentence)])
input_ids[0]

token = tokenizer(sentence, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

In [68]:
token

{'input_ids': tensor([[    0,   746,     8,    21, 46349,     5,     2,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,  

In [69]:
with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

features.last_hidden_state.size()

torch.Size([1, 7, 768])

In [72]:
with torch.no_grad():
    features = phobert(token['input_ids'])  # Models outputs are now tuples

features.last_hidden_state.size()
cls_token_embedding = features.last_hidden_state[:, 0, :]
cls_token_embedding.size()

torch.Size([1, 768])

In [4]:
classes = {
    'Công văn': 0,
    'Quyết định': 1,
    'Báo cáo': 2,
    'Thông báo': 3,
    'Kế hoạch': 4,
    'Tờ trình': 5,
    'Thư mời': 6,
    'Đơn': 7,
    'Giấy mời': 8
}

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CreateData(Dataset):
    def __init__(self, csv_file):
        self.csv_file = csv_file

        self.data_frame = pd.read_csv(csv_file, encoding='utf-8')

    def __getitem__(self, index):
        input_text = self.data_frame.loc[index, 'TRICH_YEU']
        label = self.data_frame.loc[index, 'HINHTHUC']

        input_tok, imput_msk, label = transforms_data(input_text, label)
        return input_tok, imput_msk, label
    
    def __len__(self):
        return len(self.data_frame)
    

def transforms_data(input_text, label):
    input_text = input_text.lower()
    input_text = unicodedata.normalize('NFD', input_text)
    input_text = ''.join(c for c in input_text if unicodedata.category(c) != 'Mn')
    token = tokenizer(input_text, padding='max_length', truncation=True, max_length=128, return_tensors="pt")

    input_tok = token['input_ids'][0]
    imput_msk = token['attention_mask'][0]

    label = classes[label]

    return input_tok, imput_msk, label

In [16]:
dataset = CreateData(csv_file='/home/duypd/ThisPC-DuyPC/khangta-thesis/Dataset/final_data.csv')
dataset.__len__()

101243

In [17]:
input_tok, imput_msk, label = dataset.__getitem__(78)
input_tok

tensor([  101,  2310, 20098,  2278, 27699,  2072, 24209,  6672,  2102,  1102,
         2239, 12731,  2050,  6187,  2278,  7570,  4907, 13843, 19610,  2654,
         1102, 19098,  3070,  2084,  2232, 15990,  2319,  4229,  1010, 11382,
         2368, 12835,  4048,  1102,  4887, 10722,  2002, 27468, 27793,  4017,
        16371, 10085, 22794,  2100,  1996, 14163,  5063, 27793,  4017, 16371,
        10085,  1010,  6887, 19098,  3070,  2084,  2232, 15990,  2319,  1010,
        24110,  2260,  1012,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [18]:
imput_msk

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])

In [19]:
# Chia tập dữ liệu thành 80% train và 20% validation
train_size = int(0.8 * len(dataset))
valid_size = len(dataset) - train_size
train_dataset, valid_dataset = random_split(dataset, [train_size, valid_size])

# Tạo DataLoader cho train và validation
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

In [22]:
for sample, msk, label in train_loader:
    print(sample.size())
    print(msk.size())
    print(label)
    break

torch.Size([32, 128])
torch.Size([32, 128])
tensor([2, 3, 7, 2, 3, 3, 3, 5, 1, 7, 5, 2, 7, 6, 6, 5, 2, 5, 3, 7, 1, 1, 0, 7,
        8, 5, 5, 1, 5, 1, 5, 2])
